In [3]:
!pip install requests

     |████████████████████████████████| 62 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 7.2 MB/s eta 0:00:01


In [1]:
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime

In [2]:
url_fms_arg = 'https://mundofreestyle.com/calendario-fms-argentina/'

In [3]:
response = requests.get(url=url_fms_arg)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content)

In [5]:
fms_tables_titles = soup.select('h3 strong')
fms_tables = soup.find_all(name='table')

In [6]:
[title.text for title in fms_tables_titles]

['Fechas y ciudades – Temporada 2022',
 'Temporada 2020/2021',
 'Temporada 2019',
 '¡Vive todos los eventos y fechas de FMS Argentina!']

In [7]:
fms_tables[0]

<table style="color: var( --e-global-color-text ); font-family: var( --e-global-typography-text-font-family ), Sans-serif; font-weight: var( --e-global-typography-text-font-weight ); height: 310px;" width="210">
<tbody>
<tr>
<td> </td>
<td><strong>Fecha</strong></td>
<td><strong>Ciudad</strong></td>
</tr>
<tr>
<td>1</td>
<td>4 de diciembre de 2021</td>
<td>Mendoza</td>
</tr>
<tr>
<td>2</td>
<td>29 de diciembre de 2021</td>
<td>Buenos Aires</td>
</tr>
<tr>
<td>3</td>
<td>30 de abril de 2022</td>
<td>La Plata</td>
</tr>
<tr>
<td>4</td>
<td>18 de junio de 2022</td>
<td>Rosario</td>
</tr>
<tr>
<td>5</td>
<td>27 de agosto de 2022</td>
<td>Corrientes</td>
</tr>
<tr>
<td>6</td>
<td></td><td> </td>
</tr>
<tr>
<td>7</td>
<td></td><td> </td>
</tr>
<tr>
<td>8</td>
<td></td><td> </td>
</tr>
<tr>
<td>9</td>
<td></td><td> </td>
</tr>
<tr>
<td>10</td>
<td></td><td> </td>
</tr>
<tr>
<td>11</td>
<td></td><td> </td>
</tr>
</tbody>
</table>

In [8]:
fms_tables[0].select('tr')[2].select('td')[1].text

'29 de diciembre de 2021'

In [9]:
# Testing how to convert a string date into a date type date:

test_date = str(fms_tables[0].select('tr')[2].select('td')[1].text)
test_date_clean = ",".join([word for word in test_date.split(" ") if word != 'de'])

In [17]:
locale.setlocale(locale.LC_ALL, 'es_ES.utf8')

# date_str_es = '27-Octubre-2020'  # es_ES locale
# datetime_object = datetime.strptime(date_str_es, '%d-%B-%Y')
datetime_object = datetime.strptime(test_date_clean, '%d,%B,%Y')
print(datetime_object.date())

2021-12-29


In [ ]:
def make_fms_table(soup):
    fms_tables_titles = soup.select('h3 strong')
    fms_tables = soup.find_all(name='table')
    
    # Getting the first table of each FMS league and let's convert it into a dataframe:
    fms_tables[0].select('tr')
    
    